In [6]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold,train_test_split

from sklearn.externals import joblib

nrow = 50000

In [7]:
train = pd.read_csv('../data/train.csv',nrows=nrow)
train.drop('ID_code',inplace= True,axis=1)

In [8]:
y = train['target']
x = train.drop('target',axis=1)

In [9]:
'''
model = Pipeline([
    ('scaler',MinMaxScaler()),
    ('model',LogisticRegression(C=0.01,class_weight = 'balanced',solver='lbfgs'))
]).fit(x,y)

joblib.dump(model,'../model/lr_pipe.m')
'''

"\nmodel = Pipeline([\n    ('scaler',MinMaxScaler()),\n    ('model',LogisticRegression(C=0.01,class_weight = 'balanced',solver='lbfgs'))\n]).fit(x,y)\n\njoblib.dump(model,'../model/lr_pipe.m')\n"

In [10]:
#test = pd.read_csv('../data/test.csv')

train,test = train_test_split(train,test_size = 0.3)

(15000, 201)